In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
rating_path = r'E:\School\DE_AN\Movie-Recommendation-System\src\data\ratings_small.csv'
rating_df = pd.read_csv(rating_path)
rating_df_sorted = rating_df.sort_values(by=['userId', 'timestamp'], ignore_index=True)


In [16]:
train_df, test_df = train_test_split(rating_df, \
                                         test_size=0.3, stratify=rating_df['userId'], \
                                        random_state=42)

In [18]:
test_df[test_df.userId == 1]

,userId,movieId,rating,timestamp
16,1,2294,2.0,1260759108
11,1,1405,1.0,1260759203
5,1,1263,2.0,1260759151
10,1,1371,2.5,1260759135
3,1,1129,2.0,1260759185
7,1,1293,2.0,1260759148


In [3]:
def get_train_df(df, train_size = 0.7):
    train = df.head(round(train_size * len(df)))
    return train
def get_test_df(df, train_size = 0.7):
    test = df.tail(len(df) - round((train_size) * len(df)))
    return test

In [25]:
sum(rating_df.userId.value_counts() <30)

118

In [23]:
rating_df.userId.nunique()

671

In [4]:
train_size = 0.7
train_df = pd.DataFrame(rating_df_sorted.groupby('userId').apply(lambda x: get_train_df(x, train_size)))
test_df = pd.DataFrame(rating_df_sorted.groupby('userId').apply(lambda x: get_test_df(x, train_size)))
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [5]:
print(rating_df_sorted.shape, train_df.shape, test_df.shape)

(100004, 4) (69996, 4) (30008, 4)


In [6]:
train_df[train_df['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,2294,2.0,1260759108
1,1,2455,2.5,1260759113
2,1,3671,3.0,1260759117
3,1,1339,3.5,1260759125
4,1,1343,2.0,1260759131
5,1,1371,2.5,1260759135
6,1,2105,4.0,1260759139
7,1,31,2.5,1260759144
8,1,1293,2.0,1260759148
9,1,1263,2.0,1260759151


In [7]:
test_df[test_df['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,1953,4.0,1260759191
1,1,2150,3.0,1260759194
2,1,2193,2.0,1260759198
3,1,2968,1.0,1260759200
4,1,1405,1.0,1260759203
5,1,1172,4.0,1260759205


In [8]:
title = 'Toy Story'
meta = pd.read_csv('E:\School\DE_AN\Movie-Recommendation-System\src\data\movies_metadata.csv')
links_small = pd.read_csv('E:\School\DE_AN\Movie-Recommendation-System\src\data\links_small.csv')

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_11316\67330029.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv('E:\School\DE_AN\Movie-Recommendation-System\src\data\movies_metadata.csv')


In [9]:
meta = meta.drop([19730, 29503, 35587])
meta['id'] = meta['id'].astype('int')
links_small = links_small[links_small['tmdbId'].notnull()]
links_small['tmdbId'] = links_small['tmdbId'].astype('int')
smd = meta[meta['id'].isin(links_small['tmdbId'])]
smd.shape

(9099, 24)

In [10]:
smd = smd.merge(links_small, left_on='id', right_on='tmdbId')
print(smd.shape)
smd.columns

(9099, 27)


Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'movieId', 'imdbId', 'tmdbId'],
      dtype='object')

In [11]:
movieId = smd.loc[smd.title == title]['movieId']
movieId

0    1
Name: movieId, dtype: int64

In [12]:
def get_movie_id(movie_data_df, title):
    return movie_data_df[movie_data_df.title == title]['movieId']


In [13]:
movieId

0    1
Name: movieId, dtype: int64

In [14]:
user_watch_1 = rating_df_sorted[rating_df_sorted.movieId == int(movieId.iloc[0])]['userId']
user_watch_1

495        7
729        9
911       13
1104      15
3105      19
        ... 
98531    660
98716    663
99023    664
99879    670
99951    671
Name: userId, Length: 247, dtype: int64

In [15]:
user_watch_1 = pd.Series([7, 9])
movie_recommend_id = pd.Series([10, 21, 2539])
user_watch_recommend = rating_df[(rating_df['userId'].isin(user_watch_1)) & \
    (rating_df['movieId'].isin(movie_recommend_id))]
user_watch_recommend['userId']

496    7
497    7
739    9
Name: userId, dtype: int64

In [16]:
rating_df[(rating_df['movieId'].isin(movieId))]

,userId,movieId,rating,timestamp
495,7,1,3.0,851866703
699,9,1,4.0,938629179
889,13,1,5.0,1331380058
962,15,1,2.0,997938310
3105,19,1,3.0,855190091
...,...,...,...,...
98531,660,1,2.5,1436680062
98714,663,1,4.0,1438397999
98740,664,1,3.5,1362421730
99858,670,1,4.0,938782344


In [17]:
def metric1(movie_id, movie_rec_id, rating_df):

    # users who watch movie_id
    user_movie_id = rating_df[rating_df.movieId.isin(movie_id)]['userId']
    n_user_movie_id = user_movie_id.nunique()
    
    # user watch both movie_id and movie_rec_id
    user_movie_rec_id = rating_df[(rating_df['userId'].isin(user_movie_id)) & \
                        rating_df['movieId'].isin(movie_rec_id)]['userId']
    n_user_movie_rec_id = len(user_movie_rec_id)

    return n_user_movie_rec_id / n_user_movie_id

In [18]:
movie_id = pd.Series([1])
movie_rec_id = pd.Series([10, 21])
print(metric1(movie_id, movie_rec_id, rating_df))

0.46153846153846156


In [22]:
# EVALUATE MODEL
validate = test_df[['userId', 'movieId']]
toy_val = validate[(validate.userId == 2)|(validate.userId == 1)]
toy_val

In [29]:
recommend_output = pd.DataFrame({'userId':[1,1,1,2,2,2], 'movieId':[27,1953,1172,3,550,34]})
recommend_output

,userId,movieId
0,1,27
1,1,1953
2,1,1172
3,2,3
4,2,550
5,2,34


In [59]:
recommend_output.groupby('userId').apply(lambda x: x['movieId'].isin(toy_val[toy_val['userId'] == x['userId'].iloc[0]]['movieId'])).reset_index(drop=True)

0    False
1     True
2     True
3    False
4     True
5    False
Name: movieId, dtype: bool

In [75]:
top_n = recommend_output.groupby('userId').count().iloc[0]
int(top_n.iloc[0])

3

# Test evaluation metric

In [5]:
validate_data = {'userId': [1, 1, 2, 2, 3, 3,3, 3, 3, 4, 5],
                 'movieId': [101, 102, 201, 202, 301,2, 888, 6, 7, 1, 1]}

predict_data = {'userId': [1, 1, 2, 2, 3, 3],
                'movieId': [101, 34, 202, 203, 888, 6]}

validate_df = pd.DataFrame(validate_data)
predict_df = pd.DataFrame(predict_data)
validate_df

,userId,movieId
0,1,101
1,1,102
2,2,201
3,2,202
4,3,301
5,3,2
6,3,888
7,3,6
8,3,7
9,4,1


In [6]:
predict_df

,userId,movieId
0,1,101
1,1,34
2,2,202
3,2,203
4,3,888
5,3,6


In [20]:
def check_movieId(pred_df, val_df):
    result = pred_df['movieId'].isin(val_df[val_df['userId'] == \
                                   pred_df['userId'].iloc[0]]['movieId'])
    return result

In [24]:
# Group by userId and check if movieId values in predict_df are in validate_df
# result = predict_df.groupby('userId').apply(lambda x: x['movieId'].isin(validate_df[validate_df['userId'] == x['userId'].iloc[0]]['movieId']))
# result = recommend_output.groupby('userId').apply(lambda x: check_movieId(x, toy_val))
result = predict_df.groupby('userId').apply(lambda x: check_movieId(x, validate_df))
print(result)
result = result.groupby('userId').sum()/2
print(result.groupby('userId').sum()/2)
print(result.mean())



userId   
1       0     True
        1    False
2       2     True
        3    False
3       4     True
        5     True
Name: movieId, dtype: bool
userId
1    0.25
2    0.25
3    0.50
Name: movieId, dtype: float64
0.6666666666666666


In [33]:
def evaluate(pred_df, val_df):

    result = pred_df.groupby('userId').apply(lambda x: check_movieId(x, val_df))
    result = result.groupby('userId').sum() / result.groupby('userId').count()

    return result.mean()

In [34]:
evaluate(predict_df, validate_df)

0.6666666666666666

In [31]:
def evaluate(pred_df, val_df, top_k):

    result = pred_df.groupby('userId').apply(lambda x: check_movieId(x, val_df))
    result = result.groupby('userId').sum() / top_k

    return result.mean()

In [32]:
evaluate(predict_df, validate_df, 2)

0.6666666666666666